A hobby project I took from Kaggle in 2018. The goal of the project was to find potential loan defaulters. While I started the project, I could not complete it due to work-commitments at the time. Nonetheless, I am sharing this repository for your perusal.

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime
from sklearn.preprocessing import LabelEncoder


In [3]:
train=pd.read_csv("train.csv")

In [4]:
test=pd.read_csv("test_bqCt9Pv.csv")

In [5]:
test.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES'],
      dtype='object')

In [6]:
sample=pd.read_csv("sample_submission_24jSKY6.csv")

In [7]:
len_train=len(train)

In [8]:
train=train.append(test)

In [9]:
train=train.reset_index()

In [10]:
train=train.drop('index',axis=1)

In [11]:
train.head()

,AVERAGE.ACCT.AGE,Aadhar_flag,CREDIT.HISTORY.LENGTH,Current_pincode_ID,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Date.of.Birth,DisbursalDate,Driving_flag,Employee_code_ID,Employment.Type,...,State_ID,UniqueID,VoterID_flag,asset_cost,branch_id,disbursed_amount,loan_default,ltv,manufacturer_id,supplier_id
0,0yrs 0mon,1,0yrs 0mon,1441,0,01-01-84,03-08-18,0,1998,Salaried,...,6,420825,0,58400,67,50578,0.0,89.55,45,22807
1,1yrs 11mon,1,1yrs 11mon,1502,1,31-07-85,26-09-18,0,1998,Self employed,...,6,537409,0,65550,67,47145,1.0,73.23,45,22807
2,0yrs 0mon,1,0yrs 0mon,1497,0,24-08-85,01-08-18,0,1998,Self employed,...,6,417566,0,61360,67,53278,0.0,89.63,45,22807
3,0yrs 8mon,1,1yrs 3mon,1501,0,30-12-93,26-10-18,0,1998,Self employed,...,6,624493,0,66113,67,57513,1.0,88.48,45,22807
4,0yrs 0mon,1,0yrs 0mon,1495,0,09-12-77,26-09-18,0,1998,Self employed,...,6,539055,0,60300,67,52378,1.0,88.39,45,22807


In [12]:
train['AVERAGE.ACCT.AGE']=(train['AVERAGE.ACCT.AGE'].str.slice(0,2).str.extract('(\d+)')).astype(int)*12+(train['AVERAGE.ACCT.AGE'].str.slice(3,).str.extract('(\d+)')).astype(int)

/Users/stavyabhatia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/Users/stavyabhatia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [13]:
train['CREDIT.HISTORY.LENGTH']=(train['CREDIT.HISTORY.LENGTH'].str.slice(0,2).str.extract('(\d+)')).astype(int)*12+(train['CREDIT.HISTORY.LENGTH'].str.slice(3,).str.extract('(\d+)')).astype(int)

/Users/stavyabhatia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [14]:
train['Date.of.Birth']=pd.to_datetime(train['Date.of.Birth'],format='%d-%m-%y').dt.date
train['DisbursalDate']=pd.to_datetime(train['DisbursalDate'],format='%d-%m-%y').dt.date

In [15]:
pd.set_option('display.max_columns', 60)

In [16]:
train.head()

,AVERAGE.ACCT.AGE,Aadhar_flag,CREDIT.HISTORY.LENGTH,Current_pincode_ID,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Date.of.Birth,DisbursalDate,Driving_flag,Employee_code_ID,Employment.Type,MobileNo_Avl_Flag,NEW.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,PAN_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.ACTIVE.ACCTS,PRI.CURRENT.BALANCE,PRI.DISBURSED.AMOUNT,PRI.NO.OF.ACCTS,PRI.OVERDUE.ACCTS,PRI.SANCTIONED.AMOUNT,PRIMARY.INSTAL.AMT,Passport_flag,SEC.ACTIVE.ACCTS,SEC.CURRENT.BALANCE,SEC.DISBURSED.AMOUNT,SEC.INSTAL.AMT,SEC.NO.OF.ACCTS,SEC.OVERDUE.ACCTS,SEC.SANCTIONED.AMOUNT,State_ID,UniqueID,VoterID_flag,asset_cost,branch_id,disbursed_amount,loan_default,ltv,manufacturer_id,supplier_id
0,0,1,0,1441,0,1984-01-01,2018-08-03,0,1998,Salaried,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,420825,0,58400,67,50578,0.0,89.55,45,22807
1,23,1,23,1502,1,1985-07-31,2018-09-26,0,1998,Self employed,1,0,0,0,598,I-Medium Risk,1,27600,50200,1,1,50200,1991,0,0,0,0,0,0,0,0,6,537409,0,65550,67,47145,1.0,73.23,45,22807
2,0,1,0,1497,0,1985-08-24,2018-08-01,0,1998,Self employed,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,417566,0,61360,67,53278,0.0,89.63,45,22807
3,8,1,15,1501,0,1993-12-30,2018-10-26,0,1998,Self employed,1,0,1,0,305,L-Very High Risk,0,0,0,3,0,0,31,0,0,0,0,0,0,0,0,6,624493,0,66113,67,57513,1.0,88.48,45,22807
4,0,1,0,1495,0,1977-12-09,2018-09-26,0,1998,Self employed,1,0,1,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,539055,0,60300,67,52378,1.0,88.39,45,22807


In [17]:
train=train.drop('UniqueID',axis=1)

In [18]:
train=train.drop('supplier_id',axis=1)
train=train.drop('manufacturer_id',axis=1)
train=train.drop('branch_id',axis=1)

In [19]:
train.loc[train['Employment.Type'].isnull(),'Employment.Type']='Unemployed'

In [20]:
train.Aadhar_flag=train.Aadhar_flag.astype('category')
train.Current_pincode_ID=train.Current_pincode_ID.astype('category')
train.Driving_flag=train.Driving_flag.astype('category')
train.Employee_code_ID=train.Employee_code_ID.astype('str')
train['Employment.Type']=train['Employment.Type'].astype('category')
train.MobileNo_Avl_Flag=train.MobileNo_Avl_Flag.astype('category')
train.PAN_flag=train.PAN_flag.astype('category')
train.Passport_flag=train.Passport_flag.astype('category')
train.VoterID_flag=train.VoterID_flag.astype('category')
train['PERFORM_CNS.SCORE.DESCRIPTION']=train['PERFORM_CNS.SCORE.DESCRIPTION'].astype('category')

In [21]:
future = train['Date.of.Birth'] > date(year=2050,month=1,day=1)

In [22]:
train.loc[future, 'Date.of.Birth'] -= timedelta(days=365.25*100)

In [23]:
future = train['DisbursalDate'] > date(year=2050,month=1,day=1)
train.loc[future, 'DisbursalDate'] -= timedelta(days=365.25*100)

In [24]:
train['Age']=pd.to_datetime(datetime.today().strftime('%Y-%m-%d')).date()-train['Date.of.Birth']

In [25]:
train['Age_loan']=pd.to_datetime(datetime.today().strftime('%Y-%m-%d')).date()-train['DisbursalDate']

In [26]:
train[:len_train]['loan_default'].astype(int).plot.hist()

In [27]:
train.Age=train['Age'].dt.days
train.Age_loan=train['Age_loan'].dt.days

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345546 entries, 0 to 345545
Data columns (total 39 columns):
AVERAGE.ACCT.AGE                       345546 non-null int64
Aadhar_flag                            345546 non-null category
CREDIT.HISTORY.LENGTH                  345546 non-null int64
Current_pincode_ID                     345546 non-null category
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS    345546 non-null int64
Date.of.Birth                          345546 non-null object
DisbursalDate                          345546 non-null object
Driving_flag                           345546 non-null category
Employee_code_ID                       345546 non-null object
Employment.Type                        345546 non-null category
MobileNo_Avl_Flag                      345546 non-null category
NEW.ACCTS.IN.LAST.SIX.MONTHS           345546 non-null int64
NO.OF_INQUIRIES                        345546 non-null int64
PAN_flag                               345546 non-null category
PERFORM_CNS.SC

In [29]:
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train:
    if train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train[col].unique())) <= 2:
            # Train on the training data
            le.fit(train[col])
            # Transform both training and testing data
            train[col] = le.transform(train[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

0 columns were label encoded.


In [37]:
train['year_of_birth']=pd.DatetimeIndex(train['Date.of.Birth']).year
train['year_of_disbursal']=pd.DatetimeIndex(train['DisbursalDate']).year
train['month_of_birth']=pd.DatetimeIndex(train['Date.of.Birth']).month
train['month_of_disbursal']=pd.DatetimeIndex(train['DisbursalDate']).month
train['day_of_birth']=pd.DatetimeIndex(train['Date.of.Birth']).day
train['day_of_disbursal']=pd.DatetimeIndex(train['DisbursalDate']).day

In [41]:
train=train.drop(['DisbursalDate','Date.of.Birth'],axis=1)

In [43]:
train=train.drop('Employee_code_ID',axis=1)

In [45]:
train.head()

,AVERAGE.ACCT.AGE,Aadhar_flag,CREDIT.HISTORY.LENGTH,Current_pincode_ID,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Driving_flag,Employment.Type,MobileNo_Avl_Flag,NEW.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,PAN_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.ACTIVE.ACCTS,PRI.CURRENT.BALANCE,PRI.DISBURSED.AMOUNT,PRI.NO.OF.ACCTS,PRI.OVERDUE.ACCTS,PRI.SANCTIONED.AMOUNT,PRIMARY.INSTAL.AMT,Passport_flag,SEC.ACTIVE.ACCTS,SEC.CURRENT.BALANCE,SEC.DISBURSED.AMOUNT,SEC.INSTAL.AMT,SEC.NO.OF.ACCTS,SEC.OVERDUE.ACCTS,SEC.SANCTIONED.AMOUNT,State_ID,VoterID_flag,asset_cost,disbursed_amount,loan_default,ltv,Age,Age_loan,year_of_birth,year_of_disbursal,month_of_birth,month_of_disbursal,day_of_birth,day_of_disbursal
0,0,1,0,1441,0,0,Salaried,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,58400,50578,0.0,89.55,12894,261,1984,2018,1,8,1,3
1,23,1,23,1502,1,0,Self employed,1,0,0,0,598,I-Medium Risk,1,27600,50200,1,1,50200,1991,0,0,0,0,0,0,0,0,6,0,65550,47145,1.0,73.23,12317,207,1985,2018,7,9,31,26
2,0,1,0,1497,0,0,Self employed,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,61360,53278,0.0,89.63,12293,263,1985,2018,8,8,24,1
3,8,1,15,1501,0,0,Self employed,1,0,1,0,305,L-Very High Risk,0,0,0,3,0,0,31,0,0,0,0,0,0,0,0,6,0,66113,57513,1.0,88.48,9243,177,1993,2018,12,10,30,26
4,0,1,0,1495,0,0,Self employed,1,0,1,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,60300,52378,1.0,88.39,15108,207,1977,2018,12,9,9,26


In [46]:
train = pd.get_dummies(train)


print('Training Features shape: ', train.shape)

Training Features shape:  (345546, 7163)


In [28]:
#from sklearn.preprocessing import MinMaxScaler, Imputer
#scaler = MinMaxScaler(feature_range = (0, 1))

#train = scaler.fit_transform(train)

#test=test.drop('loan_default')



In [50]:
test=train[len_train:]
train=train[:len_train]

In [51]:
test=test.drop('loan_default',axis=1)

In [52]:
y = train.loan_default
x = train.drop(columns = ['loan_default'])

In [ ]:
from sklearn.linear_model import LogisticRegression



log_reg = LogisticRegression(C = 0.0001)
log_reg.fit(x, y)

log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Train on the training data
random_forest.fit(x, y)

# Extract feature importances
#feature_importance_values = random_forest.feature_importances_
#feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

/Users/stavyabhatia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                           class_weight = 'balanced', learning_rate = 0.05, 
                           reg_alpha = 0.1, reg_lambda = 0.1, 
                           subsample = 0.8, n_jobs = -1, random_state = 50)

model.fit(x,y,eval_metric = 'auc')
lgb_predictions = model.predict_proba(test)[:,1]

In [ ]:
from mlxtend import StackCVClassifer

stack=StackCVClassifier(classifer=(random_forest,model,log_reg), meta_classifier=model, use_probas=True)
stack.fit(np.array(x),np.array(y))
stack_pred=stack.predict(np.array(test))